# EDA (ExploratoryData Analysis)

In [1]:
import polars as pl
import pandas as pd
import numpy as np

## Data Importing

In [2]:
data = pl.read_excel("data/labeled_dataset.xlsx")

In [3]:
data.show(5)

__UNNAMED__0,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
i64,str,str,str,str,str,i64,i64,str,str,str,str
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…","""usa-today""","""elections-2020""","""center""",1,1,"""Biased""","""Somewhat factual but also opin…","""YouTube says no ‘deepfakes’ or…","""['belated', 'birtherism']"""
1,"""The increasingly bitter disput…","""https://www.nbcnews.com/news/s…","""msnbc""","""sport""","""left""",1,1,"""Non-biased""","""Entirely factual""","""FRISCO, Texas — The increasing…","""['bitter']"""
2,"""So while there may be a humani…","""https://www.alternet.org/2019/…","""alternet""","""immigration""","""left""",1,1,"""Biased""","""Expresses writer’s opinion""","""Speaking to the country for th…","""['crisis']"""
3,"""A professor who teaches climat…","""https://www.breitbart.com/poli…","""breitbart""","""environment""","""right""",1,1,"""Non-biased""","""No agreement""","""A professor who teaches climat…","""['legitimate']"""
4,"""Looking around the United Stat…","""https://thefederalist.com/2020…","""federalist""","""abortion""","""right""",1,1,"""Biased""","""Somewhat factual but also opin…","""The left has a thing for takin…","""['killing', 'never', 'developi…"


In [4]:
data = data.rename(
    {
        "__UNNAMED__0": "id"
    }
)
data.show(5)

id,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
i64,str,str,str,str,str,i64,i64,str,str,str,str
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…","""usa-today""","""elections-2020""","""center""",1,1,"""Biased""","""Somewhat factual but also opin…","""YouTube says no ‘deepfakes’ or…","""['belated', 'birtherism']"""
1,"""The increasingly bitter disput…","""https://www.nbcnews.com/news/s…","""msnbc""","""sport""","""left""",1,1,"""Non-biased""","""Entirely factual""","""FRISCO, Texas — The increasing…","""['bitter']"""
2,"""So while there may be a humani…","""https://www.alternet.org/2019/…","""alternet""","""immigration""","""left""",1,1,"""Biased""","""Expresses writer’s opinion""","""Speaking to the country for th…","""['crisis']"""
3,"""A professor who teaches climat…","""https://www.breitbart.com/poli…","""breitbart""","""environment""","""right""",1,1,"""Non-biased""","""No agreement""","""A professor who teaches climat…","""['legitimate']"""
4,"""Looking around the United Stat…","""https://thefederalist.com/2020…","""federalist""","""abortion""","""right""",1,1,"""Biased""","""Somewhat factual but also opin…","""The left has a thing for takin…","""['killing', 'never', 'developi…"


In [5]:
data.select(
    pl.col("Label_opinion").value_counts()
)

Label_opinion
struct[2]
"{""No agreement"",174}"
"{""Expresses writer’s opinion"",521}"
"{""Entirely factual"",572}"
"{""Somewhat factual but also opinionated"",433}"


In [6]:
data = data.to_dummies(columns=["outlet", "topic", "type", "Label_opinion"])
data.show(1)

id,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,topic_abortion,topic_coronavirus,topic_elections-2020,topic_environment,topic_gender,topic_gun-control,topic_immigration,topic_international-politics-and-world-news,topic_middle-class,topic_sport,topic_student-debt,topic_trump-presidency,topic_vaccines,topic_white-nationalism,type_center,type_left,type_right,group_id,num_sent,Label_bias,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,biased_words4
i64,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,str,u8,u8,u8,u8,str,str
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…",0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,"""Biased""",0,0,0,1,"""YouTube says no ‘deepfakes’ or…","""['belated', 'birtherism']"""


In [7]:
import ast

parsed_lists = [ast.literal_eval(s) if s else [] for s in data["biased_words4"]]

data = data.with_columns(
    pl.Series(name="biased_words", values=parsed_lists, dtype=pl.List(pl.Utf8))
)

In [8]:
data.show(1)

id,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,topic_abortion,topic_coronavirus,topic_elections-2020,topic_environment,topic_gender,topic_gun-control,topic_immigration,topic_international-politics-and-world-news,topic_middle-class,topic_sport,topic_student-debt,topic_trump-presidency,topic_vaccines,topic_white-nationalism,type_center,type_left,type_right,group_id,num_sent,Label_bias,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,biased_words4,biased_words
i64,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,str,u8,u8,u8,u8,str,str,list[str]
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…",0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,"""Biased""",0,0,0,1,"""YouTube says no ‘deepfakes’ or…","""['belated', 'birtherism']""","[""belated"", ""birtherism""]"


In [9]:
unique_biased_words_count = (
    data.select(pl.col("biased_words"))
    .explode("biased_words")
    .unique()
    .height
)
unique_biased_words_count

2259

because of big number of unique biased words, I'm deciding to proceed with descriptive representation of this data, then data itself

In [10]:
data = data.with_columns(
    pl.col("biased_words").list.len().alias("n_biased_words"),
    (pl.col("biased_words").list.len() / unique_biased_words_count).alias("biased_words_ratio")
)

In [11]:
data.show(1)

id,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,topic_abortion,topic_coronavirus,topic_elections-2020,topic_environment,topic_gender,topic_gun-control,topic_immigration,topic_international-politics-and-world-news,topic_middle-class,topic_sport,topic_student-debt,topic_trump-presidency,topic_vaccines,topic_white-nationalism,type_center,type_left,type_right,group_id,num_sent,Label_bias,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,biased_words4,biased_words,n_biased_words,biased_words_ratio
i64,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,str,u8,u8,u8,u8,str,str,list[str],u32,f64
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…",0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,"""Biased""",0,0,0,1,"""YouTube says no ‘deepfakes’ or…","""['belated', 'birtherism']""","[""belated"", ""birtherism""]",2,0.000885


In [12]:
data = data.drop(["group_id", "num_sent", "biased_words4", "biased_words"])

In [13]:
data.show(1)

id,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,topic_abortion,topic_coronavirus,topic_elections-2020,topic_environment,topic_gender,topic_gun-control,topic_immigration,topic_international-politics-and-world-news,topic_middle-class,topic_sport,topic_student-debt,topic_trump-presidency,topic_vaccines,topic_white-nationalism,type_center,type_left,type_right,Label_bias,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,n_biased_words,biased_words_ratio
i64,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,u8,u8,u8,u8,str,u32,f64
0,"""YouTube is making clear there …","""https://eu.usatoday.com/story/…",0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"""Biased""",0,0,0,1,"""YouTube says no ‘deepfakes’ or…",2,0.000885
